In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys, os, distutils.core

!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

import torch, detectron2
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

Cloning into 'detectron2'...
remote: Enumerating objects: 15180, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 15180 (delta 111), reused 120 (delta 63), pack-reused 14979
Receiving objects: 100% (15180/15180), 6.21 MiB | 7.32 MiB/s, done.
Resolving deltas: 100% (10973/10973), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61405 sha256=6d81cf27484b444d74268bd64dd71f5da21d36ad61430dfa7bf69e8bef3368f

In [ ]:
    # # Register DeepSports dataset

    # # register_coco_instances("deepsports_train", {}, os.path.join(args.dataset_path,"train.json"),
    # #                         os.path.join(args.dataset_path,"train"))
    # # register_coco_instances("deepsports_val", {}, os.path.join(args.dataset_path,"val.json"),
    # #                         os.path.join(args.dataset_path,"val"))
    # # register_coco_instances("deepsports_test", {},
    # #                         os.path.join(args.dataset_path,"test_nolabels.json"),
    # #                         os.path.join(args.dataset_path,"test"))



    # register_coco_instances("deepsports_train", {}, "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/train.json",
    #                        "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/train")
    # register_coco_instances("deepsports_val", {}, "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/val.json",
    #                         "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/val")
    # register_coco_instances("deepsports_test", {},
    #                         "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/test_nolabels.json",
    #                         "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/test")

In [6]:
"""
Baseline code for the 2021 VIPriors Instance Segmentation challenge based on detectron2.
To use, install detectron2 () and or use the detectron2 default docker image.

Usage:
- Train baseline model:     python baseline.py --dataset_path 'path/to/dataset'
- Predict on test split:    python baseline.py --dataset_path 'path/to/dataset' --predict 'model_weights.pth.tar'

Use the last option for generating the prediction files that can be uploaded to CodaLab.

"""

# # Setup detectron2 logger
from detectron2.utils.logger import setup_logger
from detectron2.data.datasets import register_coco_instances

# import some common libraries
import torch
import os, argparse, sys, json
from tqdm import tqdm
import numpy as np
import argparse

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model
import easydict

import pycocotools.mask as mask_util

setup_logger()

def main(args):

    # Register DeepSports dataset
    register_coco_instances("deepsports_train", {}, "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/train.json",
                           "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/train")
    register_coco_instances("deepsports_val", {}, "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/val.json",
                            "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/val")
    register_coco_instances("deepsports_test", {},
                            "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/test_nolabels.json",
                            "/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022/test")

    # Load config file and adjust where needed
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.INPUT.MASK_FORMAT = 'bitmask'
    cfg.DATASETS.TRAIN = ("deepsports_train",)
    cfg.DATASETS.TEST = ("deepsports_val",)
    cfg.DATALOADER.NUM_WORKERS = 2  # 4
    cfg.SOLVER.IMS_PER_BATCH = 2 # args.bs
    cfg.SOLVER.BASE_LR = args.lr
    cfg.SOLVER.MAX_ITER = 1000 # args.iter
    cfg.SOLVER.STEPS = []
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # human (1), ball (2)
    cfg.MODEL.WEIGHTS = None # transfer learning is prohibited!
    print(cfg)

    # Create output dir
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

    if args.predict:
        # load model and weights
        model = build_model(cfg)
        DetectionCheckpointer(model).load(args.predict)
        # make dataloader
        test_loader = build_detection_test_loader(cfg, "deepsports_test")
        # predict
        outputs = []
        model.eval()
        with torch.no_grad():
            for batch in tqdm(test_loader):
                pred = model(batch)
                pred = batch_to_dict(pred, batch) # convert pred to serializable format
                outputs.extend(pred)

        with open('prediction.json', 'w') as fout:
            json.dump(outputs, fout)

        # exit
        sys.exit()

    # Train
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()
    evaluator = COCOEvaluator("deepsports_val", ("segm",), False, output_dir="./output/")
    val_loader = build_detection_test_loader(cfg, "deepsports_val")
    # print(inference_on_dataset(model, val_loader, evaluator))


In [4]:
def batch_to_dict(pred, batch):
    # TODO: check exact format to convert batch into
    results_list = []
    categories = pred[0]['instances'].pred_classes.cpu().numpy().tolist()
    scores = pred[0]['instances'].scores.cpu().numpy().tolist()
    masks = pred[0]['instances'].pred_masks.cpu()

    rles = [mask_util.encode(np.array(mask[:, :, np.newaxis], dtype=np.uint8, order="F"))[0] for mask in masks]
    for rle in rles:
        rle["counts"] = rle["counts"].decode("utf-8")

    for i in range(len(pred[0]['instances'])):
        results_list.append({'image_id': batch[0]['image_id'],
                             'category_id': categories[i]+1,
                             'segmentation': rles[i],
                             'score': scores[i]})

    return results_list

In [7]:
import easydict

if __name__ == '__main__':
    # parser = argparse.ArgumentParser(description='VIPriors Segmentation baseline training script')
    # parser.add_argument('--dataset_path', metavar='path/to/data/root', default='/content/drive/MyDrive/2023 VIPriors Instance Segmentation Challenge/vipriors-segmentation-data-2022/vipriors-segmentation-data-2022',
    #                     type=str, help='path to dataset (ends with /data)')
    # parser.add_argument('--lr', metavar='1e-4', default=1e-4, type=float, help='learning rate')
    # parser.add_argument('--bs', metavar='10', default=10, type=int, help='batch size')
    # parser.add_argument('--iter', metavar='10000', default=10000, type=int, help='training iterations')
    # parser.add_argument('--predict', default=None, type=str, help='provide model weights to run inference')
    # parser.add_argument("-f", required=False)
    # args = parser.parse_args()
    args = easydict.EasyDict({
       "dataset_path": '/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022' ,
       "lr": 1e-4 , #1e-4 -> 0.0001
       "bs": 10,
       "iter": 99,
       "predict": None })


    print(args)

    main(args)

{'dataset_path': '/content/drive/MyDrive/Colab Notebooks/PR_Lab/2023_study/vipriors-segmentation-data-2022', 'lr': 0.0001, 'bs': 10, 'iter': 99, 'predict': None}
CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 2
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('deepsports_val',)
  TRAIN: ('deepsports_train',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: bitmask
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[07/26 04:06:41 d2.utils.events]:  eta: 0:16:29  iter: 19  total_loss: 89.11  loss_cls: 26.01  loss_box_reg: 15.77  loss_mask: 2.44  loss_rpn_cls: 12.75  loss_rpn_loc: 28.51    time: 0.9920  last_time: 1.0932  data_time: 0.8328  last_data_time: 0.9449   lr: 1.9981e-06  max_mem: 2421M
[07/26 04:07:05 d2.utils.events]:  eta: 0:16:00  iter: 39  total_loss: 10.75  loss_cls: 3.822  loss_box_reg: 1.587  loss_mask: 0.7627  loss_rpn_cls: 1.197  loss_rpn_loc: 3.967    time: 1.0344  last_time: 0.6882  data_time: 0.7661  last_data_time: 0.5278   lr: 3.9961e-06  max_mem: 2421M
[07/26 04:07:26 d2.utils.events]:  eta: 0:15:59  iter: 59  total_loss: 7.725  loss_cls: 3.433  loss_box_reg: 0.6119  loss_mask: 0.691  loss_rpn_cls: 0.7381  loss_rpn_loc: 2.215    time: 1.0270  last_time: 1.0810  data_time: 0.8544  last_data_time: 0.9133   lr: 5.9941e-06  max_mem: 2421M
[07/26 04:07:47 d2.utils.events]:  eta: 0:15:49  iter: 79  total_loss: 4.922  loss_cls: 1.647  loss_box_reg: 0.3476  loss_mask: 0.6529  loss